# Planning-Lab Lesson 1: Uninformed Search Strategies

In this first lesson we will work on Uninformed Search. 

### Maze Environments
The environments used is **SmallMaze** (visible in the figure).

<img src="images/maze.png" width="300">

The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$.

In order to use the environment we need first to import the packages of OpenAI Gym. Notice that due to the structure of this repository, we need to add the parent directory to the path

In [2]:
import os, sys, time, math

module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.ai_lab_functions import *
import gym, envs

## Assignment 1: Breadth-First Search (BFS)

Your first assignment is to implement the BFS algorithm on SmallMaze. In particular, you should implement both *tree search* and *graph search* versions of BFS that the generic bfs will call. 

The results returned by your **BFS** must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - a tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory simultaneously.

**For the time_cost, we consider a node checked after its generation!**

After the correctness of your implementations has been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- BFS_TreeSearch(problem)
- BFS_GraphSearch(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for *Graph Search* and *Tree Search*:

<img src="images/tree-graph-search.png" width="600">

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for the *BFS* algorithm, note that it refers to the implementation of the *Graph Search Version*:

<img src="images/BFS2.png" width="600">

**The next two functions have to be implemented:**

In [3]:
def BFS_TreeSearch(problem):
    """
    Tree Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    # inizializza il primo nodo
    node = Node(problem.startstate, None)
    # valore incrementato ogni volta che esploro un nuovo nodo
    time_cost = 1
    # spazio occupato nella queue
    space_cost = 1
    
    # spazio totale occupato (massimo valore dello space_cost)
    max_space = 1
    
    # creo la frontiera
    frontier = NodeQueue()
    # aggiungo il nodo alla frontiera
    frontier.add(node)

    # finche` la frontiera non e' vuota    
    while not frontier.is_empty():
        # rimuovo il nodo dalla frontiera
        node = frontier.remove()
        # decremento lo spazio totale
        space_cost -= 1
        
        # esploro i nodi figli
        for action in range(problem.action_space.n):
            # trovo lo state del figlio (nella funzione problem.sample passo come parametri lo stato del padre e l'azione)
            child_state = problem.sample(node.state, action)
            # creo il nodo figlio da inserire nella queue
            child = Node(child_state, node)
            # aggiorno il time_cost
            time_cost += 1
            
            # se il nodo figlio e' la soluzione
            if child.state == problem.goalstate:
                # restituisco la soluzione
                return build_path(child), time_cost, max_space

            # altrimenti aggiungo il figlio alla queue   
            frontier.add(child)
            # aggiorno lo space cost
            space_cost += 1
            
            # se lo space_ cost e' maggiore del max_space
            if space_cost > max_space:
                # aggiorno il max_space con il valore dello space_cost
                max_space = space_cost
    
    # nel caso la queue sia vuota restituisco la soluzione con il goal_state non trovato
    return build_path(node), time_cost, max_space

In [4]:
def BFS_GraphSearch(problem):
    """
    Graph Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    # nodo di partenza del problema
    node = Node(problem.startstate, None)
    # tempo richiusto (aumenta a ogni esplorazione del nodo) 
    time_cost = 1
    # aumenta o diminuisce a seconda della grandezza della frontier
    space_cost = 1
    # spazio massimo occupato dai nodi in frontier
    max_space = 1

    # frontiera e' una queue
    frontier = NodeQueue()
    # inizializzata con il nodo di partenza
    frontier.add(node)
    # nel GraphSearch si tiene conto dei nodi esplorati senza ripetizioni
    explored = set()

    # ciclo finche' la frontiera non e' vuota    
    while not frontier.is_empty():
        # nodo preso in esame, come per il treeSearch, e' il primo nodo rimosso dalla frontiera
        node = frontier.remove()
        # viene aggiunto lo stato dei nodi in explored  -> serve per l'aggiunta dei nodi da esplorare nella frontiera
        explored.add(node.state)

        # ciclo per le 4 azioni che posso copiere         
        for action in range(problem.action_space.n):
            # mi salvo lo stato del nodo figlio
            child_state = problem.sample(node.state, action)
            # mi salvo il nodo
            child = Node(child_state, node)
            time_cost += 1
            
            # se il nodo figlio e' proprio la soluzione al mio problema
            if child.state == problem.goalstate:
                # restituisci la soluzione
                return build_path(child), time_cost, max_space
            
            # se il nodo figlio non fa parte dei nodi esplorati e non e' gia' presente nella frontiera
            if child.state not in explored and child.state not in frontier:
                # aggiungi alla frontiera il nodo figlio
                frontier.add(child)
                # aumenta lo space cost
                space_cost += 1
                
                # se lo space cost e' maggiore del max_space -> aggiorna max_space
                if space_cost > max_space:
                    max_space = space_cost
    
    return build_path(node), time_cost, max_space

**The following code calls your tree search and graph search version of BFS and checks the results**

In [5]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = BFS_TreeSearch(environment)
solution_gs, time_gs, memory_gs = BFS_GraphSearch(environment)

results = CheckResult_L1A1([solution_ts, time_ts, memory_ts], [solution_gs, time_gs, memory_gs], environment)
results.check_sol_ts()
results.check_sol_gs()

##########################################
#######  BFS TREE SEARCH PROBLEM  ########
##########################################
Your solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 103723
Max n° of nodes in memory: 77791

===> Your solution is correct!

##########################################
#######  BFS Graph SEARCH PROBLEM  #######
##########################################
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 59
Max n° of nodes in memory: 15

===> Your solution is correct!



## Assignment 2:  Depth-Limited Search (DLS) and Iterative Deepening depth-first Search (IDS)

Your second assignment is to implement the IDS algorithm on SmallMaze. 
In particular, you are required to implement *DLS* in the *graph search* version, *DLS* in the *tree search* version, and the final *Iterative_DLS*.

Similarly to assignment 1, the results returned by your ids must be in the following form (path, Time Cost, Space Cost) described above. After the correctness of your implementations has been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- Recursive_DLS_TreeSearch(node, problem, limit)
- Recursive_DLS_GraphSearch(node, problem, limit, explored)
- IDS(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for the *Depth-Limited Search (Tree Search Version)* and *Iterative deepening depth-first search (Tree Search Version)*:
<img src="images/dls.png" width="600">
<img src="images/ids.png" width="600">

#### Note that Node() has a depthcost variable that represents the depth of the node in the search tree. This variable is automatically set by the Node constructor: if the root node has a depthcost of 0, its children will have a depthcost increased by 1. An example is shown below. The depthcost is useful to compute the space cost for IDS. An example is shown below:

In [6]:
start = environment.startstate
root = Node(start)  # parent = None and depthcost = 0 as default


child = Node(environment.sample(start, 0), root) # the depthcost is set automatically in the Node constructor
print("Root depthcost: {}\tChild depthcost: {}".format(root.depthcost, child.depthcost))

Root depthcost: 0	Child depthcost: 1


In [7]:
def DLS(problem, limit, RDLS_Function):
    """
    DLS
    
    Args:
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
       
    node = Node(problem.startstate, None)
    result = RDLS_Function(node, problem, limit, set())
    return result[0], result[1] + 1, result[2]

**The next two functions have to be implemented:**

In [8]:
def Recursive_DLS_GraphSearch(node, problem, limit, explored):
    """
    Recursive DLS
    
    Args:
        node: node to explore
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        explored: completely explored nodes
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    # N.B. SIMILE AL BFS MA ABBIAMO UN LIMITE DA NON SUPERARE
    # costruito in maniera ricorsiva


    # casi base:
    # se lo stato del nodo == goal -> restituisci la soluzione
    if node.state == problem.goalstate:
        return build_path(node), 0, node.depthcost
    # se ho raggiunto il limite senza trovare la soluzione -> "cutoff" message, 0, profondita' albero
    elif limit == 0:
        return "cut_off", 0, node.depthcost
    # chiamata ricorsiva
    else:
        # aggiungere il nuovo nodo ad explored
        explored.add(node.state)
        
        # il cutoff non e' occorso
        cutoff_occurred = False
        
        time_cost = 0
        space_cost = 0
        
        # controllo tutte le azioni nel range
        for action in range(problem.action_space.n):
            child_state = problem.sample(node.state, action)
            
            if child_state in explored:
                continue
            
            # creo il nodo figlio
            child = Node(child_state, node)
            time_cost += 1
            
            # richiamo la funzione DLS decrementando il limite
            result = Recursive_DLS_GraphSearch(child, problem, limit - 1, explored)
            time_cost += result[1]
            space_cost = max(space_cost, result[2])
            
            # se ho raggiunto il cutoff -> flag true
            if result[0] == "cut_off":
                cutoff_occurred = True
            # restituisco il risultato in caso di successo
            elif result[0] != "failure":
                return result[0], time_cost, space_cost
        
        if cutoff_occurred:
            return "cut_off", time_cost, space_cost
        else:
            return "failure", time_cost, space_cost

In [9]:
def Recursive_DLS_TreeSearch(node, problem, limit, explored=None):
    """
    DLS (Tree Search Version)
    
    Args:
        node: node to explore
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    # tree-search ma costruito in maniera ricorsiva
    
    # se lo stato del nodo e' il goal state
    if node.state == problem.goalstate:
        # restituisci la soluzione
        return build_path(node), 0, node.depthcost
    # se viene raggiunto il limite
    elif limit == 0:
        # restituisci la solzione con al posto del percorso la scritta "cut-off"
        return "cut_off", 0, node.depthcost
    # se il cutoff non e' occorso
    else:
        cutoff_occurred = False
        
        # inizializzazione time_cost e space_cost
        time_cost = 0
        space_cost = 0
        
        # esploro i nodi figli
        for action in range(problem.action_space.n):
            # creo il nodo figlio
            child_state = problem.sample(node.state, action)
            # "creo" il nodo figlio
            child = Node(child_state, node)
            
            # incremento il time cost
            time_cost += 1
            
            # chiamata ricorsiva del risultato passando come terzo parametro il limite-1
            result = Recursive_DLS_TreeSearch(child, problem, limit - 1, explored)
            # il time cost viene calcolato sommando tutti i problem
            time_cost += result[1]
            # space cost ottenuto trovando il massimo tra space cost e limit
            space_cost = max(space_cost, result[2])
            
            # se il child e' cutoff
            if result[0] == "cut_off":
                # si pone a vero il cutoff
                cutoff_occurred = True
            # altrimenti non si e' trovata la soluzione
            elif result[0] != "failure":
                return result[0], time_cost, space_cost
        
        if cutoff_occurred:
            return "cut_off", time_cost, space_cost
        else:
            return "failure", time_cost, space_cost
        

In [10]:
def IDS(problem, DLS_Function):
    """
    Iterative_DLS DLS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
        
    total_time_cost = 0
    total_space_cost = 1
    
    for i in zero_to_infinity():
        result = DLS(problem, i, DLS_Function)
        
        total_time_cost += result[1]
        total_space_cost = max(total_space_cost, result[2])
        
        if result[0] != "cut_off":
            return result[0], total_time_cost, total_space_cost, i
        
    return solution_dls, total_time_cost, total_space_cost, i

**The following code calls your version of IDS and checks the results:**

In [11]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts, iterations_ts = IDS(environment, Recursive_DLS_TreeSearch)
solution_gs, time_gs, memory_gs, iterations_gs = IDS(environment, Recursive_DLS_GraphSearch)

results = CheckResult_L1A2([solution_ts, time_ts, memory_ts, iterations_ts], [solution_gs, time_gs, memory_gs, iterations_gs], environment)
results.check_sol_ts()
results.check_sol_gs()

##########################################
#######  IDS TREE SEARCH PROBLEM  ########
##########################################
Necessary Iterations: 9
Your solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 138298
Max n° of nodes in memory: 9

===> Your solution is correct!

##########################################
#######  IDS GRAPH SEARCH PROBLEM  #######
##########################################
Necessary Iterations: 11
Solution: [(0, 1), (0, 0), (1, 0), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 132
Max n° of nodes in memory: 11

===> Your solution is correct!



### Discussion

Now that you have correctly implemented both BFS and IDS what can you say about the solutions they compute? Are there significant differences in the stats?